In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from sklearn import #something

raw_data = pd.read_csv('Absenteeism_data.csv')
df_hr = raw_data.copy()

# pd.options.display.max_rows = None
# pd.options.display.max_columns = None

# A couple of takeaways here, the date format is not in the typical mm/dd/yyyy formatI it's dd/mm/yyyy
# The 'Absenteeism Time in Hours' column will make a good 'target' feature for machine learning
# Will probably use a regression model after pre-processing


df_hr.head()

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [2]:
def null_columns(df):
    return(pd.isnull(df).sum())

# This dataset has no missing values, a rarity.
print("The data set has", df_hr.shape[0], "rows and", df_hr.shape[1], "columns")
print(null_columns(df_hr))

The data set has 700 rows and 12 columns
ID                           0
Reason for Absence           0
Date                         0
Transportation Expense       0
Distance to Work             0
Age                          0
Daily Work Load Average      0
Body Mass Index              0
Education                    0
Children                     0
Pets                         0
Absenteeism Time in Hours    0
dtype: int64


In [3]:
# looking for anomalies:
# Will explore 'Reason for Absence' oclumn. Looks to be the primary independent variable
df_hr.describe()

,ID,Reason for Absence,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
count,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,17.951429,19.411429,222.347143,29.892857,36.417143,271.801774,26.737143,1.282857,1.021429,0.687143,6.761429
std,11.028144,8.356292,66.312960,14.804446,6.379083,40.021804,4.254701,0.668090,1.112215,1.166095,12.670082
min,1.000000,0.000000,118.000000,5.000000,27.000000,205.917000,19.000000,1.000000,0.000000,0.000000,0.000000
25%,9.000000,13.000000,179.000000,16.000000,31.000000,241.476000,24.000000,1.000000,0.000000,0.000000,2.000000
50%,18.000000,23.000000,225.000000,26.000000,37.000000,264.249000,25.000000,1.000000,1.000000,0.000000,3.000000
75%,28.000000,27.000000,260.000000,50.000000,40.000000,294.217000,31.000000,1.000000,2.000000,1.000000,8.000000
max,36.000000,28.000000,388.000000,52.000000,58.000000,378.884000,38.000000,4.000000,4.000000,8.000000,120.000000


# Pre-processing / Data Cleaning

In [4]:
# Dropping ID column (unnecessary)
df_hr = df_hr.drop(['ID'], axis = 1)

In [5]:
# Exploring 'Reason for abscence' column:
# This feature appears to be categorical nominal ; that is, it's used to categorize using numbers instead of names.

print('Min value: ', df_hr['Reason for Absence'].min())
print('Max value: ', df_hr['Reason for Absence'].max())
print('Length: ', len(df_hr['Reason for Absence'].unique()))

# There appears to be a missing value (or, a particular coded absence reason hasn't been used)
# Will need to locate it
# The missing reason number is 20
print('Missing number: ', [i for i in range(0,29) if i not in sorted(df_hr['Reason for Absence'].unique())])

Min value:  0
Max value:  28
Length:  28
Missing number:  [20]


In [6]:
# I'm going to one-hot encode this column into dummy variables.
# This will allow deeper analysis for each individual reason.
# This also assumes that an individual can only be absent for one reason, and not multiple


# Dropping reason 0 to avoid multicolinearity. 
# This is important because two or more variables that are dependant on other variables
# can be predicted with substantial accuracy. 
absent_reasons = pd.get_dummies(df_hr['Reason for Absence'], drop_first=True)
absent_reasons

,1,2,3,4,5,6,7,8,9,10,...,18,19,21,22,23,24,25,26,27,28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
696,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
697,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


## Bucketing/Concatenating

In [7]:
# Separating individual reasons into larger 'buckets' to reduce cardinality (see image in repo)

# The first 14 reasons are all related to various diseases
# Reasons 15-17 are related to pregnancy
# Reasons 18-21 are related to poisoning or otherwise unique circumstances
# The rest are more 'lightweight' (typically non-life threatening) reasons for absence
# such as a dental appointments, therapy, or medical consultations
reason_type_1 = absent_reasons.iloc[:,:14].max(axis=1)
reason_type_2 = absent_reasons.iloc[:,14:17].max(axis=1)
reason_type_3 = absent_reasons.iloc[:, 17:21].max(axis=1)
reason_type_4 = absent_reasons.iloc[:, 21:].max(axis=1)

In [8]:
# dropping reason for absence column because the four absence reasons replace the need for it
df_hr = df_hr.drop(['Reason for Absence'], axis=1)

# Concatenating reason buckets back into main dataframe
df_hr = pd.concat([df_hr, reason_type_1, reason_type_2, reason_type_3, reason_type_4], axis=1)

df_hr.head()

,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,0,1,2,3
0,07/07/2015,289,36,33,239.554,30,1,2,1,4,0,0,0,1
1,14/07/2015,118,13,50,239.554,31,1,1,0,0,0,0,0,0
2,15/07/2015,179,51,38,239.554,31,1,0,0,2,0,0,0,1
3,16/07/2015,279,5,39,239.554,24,1,2,0,4,1,0,0,0
4,23/07/2015,289,36,33,239.554,30,1,2,1,2,0,0,0,1


In [9]:
# Renaming reason column names for readability
df_hr.columns.values

array(['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours', 0, 1, 2, 3],
      dtype=object)

In [10]:
# Renaming reason column names for readability

column_names = ['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours','Reason_1','Reason_2','Reason_3','Reason_4']

df_hr.columns = column_names
df_hr.head()

,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Reason_1,Reason_2,Reason_3,Reason_4
0,07/07/2015,289,36,33,239.554,30,1,2,1,4,0,0,0,1
1,14/07/2015,118,13,50,239.554,31,1,1,0,0,0,0,0,0
2,15/07/2015,179,51,38,239.554,31,1,0,0,2,0,0,0,1
3,16/07/2015,279,5,39,239.554,24,1,2,0,4,1,0,0,0
4,23/07/2015,289,36,33,239.554,30,1,2,1,2,0,0,0,1


## Re-order Columns

In [11]:
# Moving reasons to the front of the table, just like the original column was
column_names_reordered = ['Reason_1','Reason_2','Reason_3','Reason_4','Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours']
df_hr = df_hr[column_names_reordered]

df_hr.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [12]:
df_reason_mod = df_hr.copy()

df_reason_mod.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,23/07/2015,289,36,33,239.554,30,1,2,1,2


## Date

In [13]:
df_reason_mod['Date']

0      07/07/2015
1      14/07/2015
2      15/07/2015
3      16/07/2015
4      23/07/2015
          ...    
695    23/05/2018
696    23/05/2018
697    24/05/2018
698    24/05/2018
699    31/05/2018
Name: Date, Length: 700, dtype: object

In [14]:
# The date column is isn't in datetime format. It's also not mm/dd/yyyy syntax. It needs to be cleaned
df_reason_mod.dtypes

Reason_1                       uint8
Reason_2                       uint8
Reason_3                       uint8
Reason_4                       uint8
Date                          object
Transportation Expense         int64
Distance to Work               int64
Age                            int64
Daily Work Load Average      float64
Body Mass Index                int64
Education                      int64
Children                       int64
Pets                           int64
Absenteeism Time in Hours      int64
dtype: object

In [15]:
# Changing date column
df_reason_mod['Date'] = pd.to_datetime(df_reason_mod['Date'], format='%d/%m/%Y')
df_reason_mod['Date']

0     2015-07-07
1     2015-07-14
2     2015-07-15
3     2015-07-16
4     2015-07-23
         ...    
695   2018-05-23
696   2018-05-23
697   2018-05-24
698   2018-05-24
699   2018-05-31
Name: Date, Length: 700, dtype: datetime64[ns]

In [16]:
# Encoding values from date column. Separating out month and dayto a separate column to improve analysis
df_reason_mod['Month'] = df_reason_mod['Date'].dt.month

# Now for weekdays
df_reason_mod['Weekday'] = df_reason_mod['Date'].dt.weekday

# Dropping date column. It's no longer needed
df_reason_mod.drop(['Date'], axis=1)

df_reason_mod.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Month,Weekday
0,0,0,0,1,2015-07-07,289,36,33,239.554,30,1,2,1,4,7,1
1,0,0,0,0,2015-07-14,118,13,50,239.554,31,1,1,0,0,7,1
2,0,0,0,1,2015-07-15,179,51,38,239.554,31,1,0,0,2,7,2
3,1,0,0,0,2015-07-16,279,5,39,239.554,24,1,2,0,4,7,3
4,0,0,0,1,2015-07-23,289,36,33,239.554,30,1,2,1,2,7,3


## The other features

The rest of the features are either integers or floats. They are also pretty intuitive:

- Transportation expense (in dollars)
- Age
- Daily work load average (in minutes)
- Body Mass Index - a reasonable indicator of health with regards to weight
- Pets - Categorical numeric
- Children - Categorical numeric

The above features will not be cleaned, but the next features will be examined more closely:

- Education

This is categorical nominal, and deserves closer examination

The education values correspond to education levels:

- 1 = High school
- 2 = Graduate
- 3 = Postgraduate
- 4 = Masters or Doctorate


In [17]:
# 83% of all people in the this data set have a high school education only
df_reason_date_mod = df_reason_mod.copy()

df_reason_date_mod['Education'].value_counts() / df_reason_date_mod.shape[0]

1    0.832857
3    0.104286
2    0.057143
4    0.005714
Name: Education, dtype: float64

In [18]:
# It'd make sense to bin the education levels by high school and beyond.
# Separating by level of graduate degree is now less important
# Mapping high school to 0 and beyond to 1, making this column binary

df_reason_date_mod['Education'] = df_reason_date_mod['Education'].map({1:0, 2:1, 3:1, 4:1})

# Verifying binning process
df_reason_date_mod['Education'].value_counts()

0    583
1    117
Name: Education, dtype: int64

# Machine Learning